In [55]:
import pandas as pd
from pathlib import Path

FILE_PATH = Path("ITU_regional_global_Key_ICT_indicator_aggregates_Nov_2024.xlsx")
OUTPUT_DIR = Path("./output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


def is_year(x):
    try:
        xi = int(float(x))
        return 1990 <= xi <= 2035
    except Exception:
        return False


def clean_number(v):
    if pd.isna(v):
        return None
    s = str(v).strip()
    if s == "" or s.upper() in {"N/A", "NA"}:
        return None
    s = s.replace(" ", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return None


def read_sheet_drop_first_row(path: Path, sheet_name: str) -> pd.DataFrame:
    df = pd.read_excel(path, sheet_name=sheet_name, header=None, engine="openpyxl")
    if len(df) >= 1:
        df = df.iloc[1:].reset_index(drop=True)
    return df


def find_header_row_with_years(df: pd.DataFrame, start_row=0, max_scan=160) -> int | None:
    nrows = len(df)
    end_row = min(start_row + max_scan, nrows)
    for r in range(start_row, end_row):
        row_vals = df.iloc[r].tolist()
        if sum(1 for v in row_vals if is_year(v)) >= 3:
            return r
    return None


# ----------- Critères figés -----------

CRITERIA_BY_GENDER = [
    "Percentage of individuals using the Internet, by gender",
    "Percentage of individuals owning a mobile phone*, by gender",
]

CRITERIA_BY_AGE = [
    "Percentage of individuals using the Internet, by age",
]

CRITERIA_URBAN_RURAL = [
    "Population covered by a mobile-cellular network (%)",
    "Population covered by at least a 3G mobile network (%)",
    "Population covered by at least an LTE/WiMAX mobile network (%)",
    "Population covered by at least a 5G mobile network (%)",
    "Percentage of individuals using the Internet, by urban/rural area",
]

CRITERIA_BDT_REGION = [
    "Fixed-telephone subscriptions/Millions",
    "Fixed-broadband subscriptions/Millions",
    "Mobile-cellular telephone subscriptions/Millions",
    "Active mobile-broadband subscriptions/Millions",
    "Population covered by a mobile-cellular network/Millions",
    "Population covered by at least a 3G mobile network/Millions",
    "Population covered by at least an LTE/WiMAX mobile network/Millions",
    "Population covered by at least a 5G mobile network/Millions",
    "Fixed broadband traffic(Exabytes)",
    "Mobile broadband traffic(Exabytes)",
    "International bandwidth usage(Tbits/s**)",
    "Individuals using the Internet(Millions)",
    "Individuals owning a mobile phone***(Millions)",
]

CRITERIA_DVPMNT_SPEC = [
    "Fixed-telephone subscriptions",
    "Fixed-broadband subscriptions",
    "Mobile-cellular telephone subscriptions",
    "Active mobile-broadband subscriptions",
    "Population covered by a mobile-cellular network",
    "Population covered by at least a 3G mobile network",
    "Population covered by at least an LTE/WiMAX mobile network",
    "Population covered by at least a 5G mobile network",
    "Fixed broadband traffic",
    "Mobile broadband traffic",
    "International bandwidth usage",
    "Individuals using the Internet",
    "Individuals owning a mobile phone***",
]


# ----------- melt par segments de colonnes (catégorie au-dessus) -----------

def melt_block_with_column_categories(df: pd.DataFrame, header_row: int) -> pd.DataFrame:
    """
    Pour une ligne d'années header_row, trouve les segments contigus de colonnes d'années
    (Total, Female, Urban, etc.), lit la catégorie sur la ligne h-1 au-dessus de la
    première colonne de chaque segment, et renvoie un DataFrame concaténé:
    Region, Year, Value, Category.
    """
    nrows, ncols = df.shape
    # Colonnes d'années
    year_cols = [c for c, v in enumerate(df.iloc[header_row]) if is_year(v)]
    if not year_cols:
        return pd.DataFrame()

    # Regroupe les colonnes d'années en segments contigus
    segments = []
    start = year_cols[0]
    prev = start
    for c in year_cols[1:]:
        if c == prev + 1:
            prev = c
        else:
            segments.append((start, prev))
            start = c
            prev = c
    segments.append((start, prev))

    blocks = []
    data_start = header_row + 1

    for (c_start, c_end) in segments:
        # Catégorie : texte sur la ligne juste au-dessus de c_start
        cat = "Total"
        if header_row > 0:
            r_cat = header_row - 1
            val = df.iat[r_cat, c_start]
            if not (isinstance(val, str) and val.strip()) and c_start + 1 < ncols:
                # parfois le texte est dans la colonne juste à droite
                val2 = df.iat[r_cat, c_start + 1]
                if isinstance(val2, str) and val2.strip():
                    val = val2
            if isinstance(val, str) and val.strip():
                cat = val.strip()

        # Sous-DataFrame : Region + ce segment de colonnes d'années
        cols = [0] + list(range(c_start, c_end + 1))
        sub = df.iloc[data_start:, cols].copy()
        # Filtre les lignes vides côté Region
        sub = sub[
            ~sub.iloc[:, 0].isna()
            & (sub.iloc[:, 0].astype(str).str.strip() != "")
        ]
        if sub.empty:
            continue

        years = [
            int(float(df.iat[header_row, c])) for c in range(c_start, c_end + 1)
        ]
        sub.columns = ["Region"] + years
        melted = sub.melt(id_vars=["Region"], var_name="Year", value_name="Value")
        melted["Value"] = melted["Value"].map(clean_number)
        melted = melted.dropna(subset=["Value"])
        if melted.empty:
            continue
        melted["Category"] = cat
        blocks.append(melted)

    if not blocks:
        return pd.DataFrame()

    out = pd.concat(blocks, ignore_index=True)
    return out


# ----------- extraction séquentielle de headers -----------

def extract_blocks_sequentially(df: pd.DataFrame, max_headers: int):
    """
    Parcourt le DataFrame pour trouver des lignes d'années successives.
    Pour chaque ligne d'années, applique melt_block_with_column_categories.
    Retourne une liste de DataFrames (chaque DF contient déjà Category).
    """
    parts = []
    cursor = 0
    found = 0
    while found < max_headers:
        h = find_header_row_with_years(df, start_row=cursor, max_scan=200)
        if h is None:
            break
        block = melt_block_with_column_categories(df, h)
        if block.empty:
            cursor = h + 1
            continue
        parts.append(block)
        found += 1
        cursor = h + 5
    return parts


def assign_criteria_in_order(parts, criteria, headers_per_criterion=1):
    """
    Associe chaque critère à une ou plusieurs lignes d'années (headers).

    parts : liste de DataFrames retournés par extract_blocks_sequentially
            (chaque élément = toutes les catégories pour UN header_row).
    criteria : liste de critères pour la feuille.
    headers_per_criterion : nombre de lignes d'années par critère (souvent 1).
    """
    out_parts = []
    expected_headers = len(criteria) * headers_per_criterion
    use_parts = parts[:expected_headers]
    for i, block in enumerate(use_parts):
        crit_idx = i // headers_per_criterion
        b = block.copy()
        b["Criteria"] = criteria[crit_idx]
        out_parts.append(b)
    return out_parts


# ----------- Fonctions par feuille -----------

def extract_by_gender():
    xls = pd.ExcelFile(FILE_PATH)
    sheet = next((s for s in xls.sheet_names if "by gender" in s.lower()), None)
    if sheet is None:
        raise ValueError("Feuille 'By gender' introuvable")
    df = read_sheet_drop_first_row(FILE_PATH, sheet)

    # 2 critères (Internet, mobile phone). En pratique: 1 header par critère
    header_parts = extract_blocks_sequentially(df, max_headers=len(CRITERIA_BY_GENDER))
    parts = assign_criteria_in_order(header_parts, CRITERIA_BY_GENDER, headers_per_criterion=1)

    if not parts:
        raise ValueError("Aucun bloc reconnu dans 'By gender'")

    out = pd.concat(parts, ignore_index=True)[
        ["Region", "Year", "Category", "Value", "Criteria"]
    ]
    out.to_csv(OUTPUT_DIR / "gender_flat.csv", index=False)
    print("OK ->", OUTPUT_DIR / "gender_flat.csv")


def extract_by_age():
    xls = pd.ExcelFile(FILE_PATH)
    sheet = next((s for s in xls.sheet_names if "internet use by age" in s.lower()), None)
    if sheet is None:
        raise ValueError("Feuille 'Internet use by age' introuvable")
    df = read_sheet_drop_first_row(FILE_PATH, sheet)

    # 1 critère, 1 header
    header_parts = extract_blocks_sequentially(df, max_headers=1)
    parts = assign_criteria_in_order(header_parts, CRITERIA_BY_AGE, headers_per_criterion=1)

    if not parts:
        raise ValueError("Aucun bloc reconnu dans 'Internet use by age'")

    out = pd.concat(parts, ignore_index=True)[
        ["Region", "Year", "Category", "Value", "Criteria"]
    ]
    out.to_csv(OUTPUT_DIR / "age_flat.csv", index=False)
    print("OK ->", OUTPUT_DIR / "age_flat.csv")


def extract_by_urban_rural_area():
    xls = pd.ExcelFile(FILE_PATH)
    sheet = next((s for s in xls.sheet_names if "urban-rural" in s.lower()), None)
    if sheet is None:
        raise ValueError("Feuille 'By urban-rural area' introuvable")
    df = read_sheet_drop_first_row(FILE_PATH, sheet)

    # 5 critères, 1 header par critère
    header_parts = extract_blocks_sequentially(df, max_headers=len(CRITERIA_URBAN_RURAL))
    parts = assign_criteria_in_order(header_parts, CRITERIA_URBAN_RURAL, headers_per_criterion=1)

    if not parts:
        raise ValueError("Aucun bloc reconnu dans 'By urban-rural area'")

    out = pd.concat(parts, ignore_index=True)[
        ["Region", "Year", "Category", "Value", "Criteria"]
    ]
    out.to_csv(OUTPUT_DIR / "urban_rural_flat.csv", index=False)
    print("OK ->", OUTPUT_DIR / "urban_rural_flat.csv")


def extract_by_bdt_region():
    xls = pd.ExcelFile(FILE_PATH)
    sheet = next((s for s in xls.sheet_names if "bdt region" in s.lower()), None)
    if sheet is None:
        raise ValueError("Feuille 'By BDT region' introuvable")
    df = read_sheet_drop_first_row(FILE_PATH, sheet)

    # 13 critères, 1 header par critère
    header_parts = extract_blocks_sequentially(df, max_headers=len(CRITERIA_BDT_REGION))
    parts = assign_criteria_in_order(header_parts, CRITERIA_BDT_REGION, headers_per_criterion=1)

    if not parts:
        raise ValueError("Aucun bloc reconnu dans 'By BDT region'")

    out = pd.concat(parts, ignore_index=True)[
        ["Region", "Year", "Category", "Value", "Criteria"]
    ]
    out.to_csv(OUTPUT_DIR / "bdt_region_flat.csv", index=False)
    print("OK ->", OUTPUT_DIR / "bdt_region_flat.csv")


def extract_by_dvpmnt_status_and_spec_reg():
    xls = pd.ExcelFile(FILE_PATH)
    sheet = next(
        (s for s in xls.sheet_names if "dvpmnt status" in s.lower() or "spec. reg" in s.lower()),
        None,
    )
    if sheet is None:
        raise ValueError("Feuille 'By dvpmnt status and spec. reg.' introuvable")
    df = read_sheet_drop_first_row(FILE_PATH, sheet)

    # 13 critères, 1 header par critère
    header_parts = extract_blocks_sequentially(df, max_headers=len(CRITERIA_DVPMNT_SPEC))
    parts = assign_criteria_in_order(header_parts, CRITERIA_DVPMNT_SPEC, headers_per_criterion=1)

    if not parts:
        raise ValueError("Aucun bloc reconnu dans 'By dvpmnt status and spec. reg.'")

    out = pd.concat(parts, ignore_index=True)[
        ["Region", "Year", "Category", "Value", "Criteria"]
    ]
    out.to_csv(OUTPUT_DIR / "dvpmnt_status_spec_reg_flat.csv", index=False)
    print("OK ->", OUTPUT_DIR / "dvpmnt_status_spec_reg_flat.csv")


if __name__ == "__main__":
    extract_by_gender()
    extract_by_age()
    extract_by_urban_rural_area()
    extract_by_bdt_region()
    extract_by_dvpmnt_status_and_spec_reg()

OK -> output\gender_flat.csv
OK -> output\age_flat.csv
OK -> output\urban_rural_flat.csv
OK -> output\bdt_region_flat.csv
OK -> output\dvpmnt_status_spec_reg_flat.csv


In [56]:
import pandas as pd

In [57]:
df_gender = pd.read_csv("output\gender_flat.csv")
df_gender

,Region,Year,Category,Value,Criteria
0,World,2019,Total,52.9,"Percentage of individuals using the Internet, ..."
1,Low-income,2019,Total,17.2,"Percentage of individuals using the Internet, ..."
2,Lower-middle-income,2019,Total,33.5,"Percentage of individuals using the Internet, ..."
3,Upper-middle-income,2019,Total,64.5,"Percentage of individuals using the Internet, ..."
4,High-income,2019,Total,87.5,"Percentage of individuals using the Internet, ..."
...,...,...,...,...,...
635,Americas,2024,Total,85.5,Percentage of individuals owning a mobile phon...
636,Arab States,2024,Total,75.1,Percentage of individuals owning a mobile phon...
637,Asia-Pacific,2024,Total,67.5,Percentage of individuals owning a mobile phon...
638,CIS,2024,Total,92.6,Percentage of individuals owning a mobile phon...


In [58]:
def_age = pd.read_csv(r"output\age_flat.csv")
def_age

,Region,Year,Category,Value,Criteria
0,World,2021,Total,61.7,"Percentage of individuals using the Internet, ..."
1,Low-income,2021,Total,20.9,"Percentage of individuals using the Internet, ..."
2,Lower-middle-income,2021,Total,46.6,"Percentage of individuals using the Internet, ..."
3,Upper-middle-income,2021,Total,74.1,"Percentage of individuals using the Internet, ..."
4,High-income,2021,Total,90.0,"Percentage of individuals using the Internet, ..."
...,...,...,...,...,...
163,Americas,2024,Rest of the population,85.1,"Percentage of individuals using the Internet, ..."
164,Arab States,2024,Rest of the population,66.7,"Percentage of individuals using the Internet, ..."
165,Asia-Pacific,2024,Rest of the population,63.5,"Percentage of individuals using the Internet, ..."
166,CIS,2024,Rest of the population,90.7,"Percentage of individuals using the Internet, ..."


In [59]:
df_urban_rural = pd.read_csv(r"output\urban_rural_flat.csv")
df_urban_rural

,Region,Year,Category,Value,Criteria
0,World,2015,Total,94.9,Population covered by a mobile-cellular networ...
1,Low-income,2015,Total,80.7,Population covered by a mobile-cellular networ...
2,Lower-middle-income,2015,Total,94.4,Population covered by a mobile-cellular networ...
3,Upper-middle-income,2015,Total,96.7,Population covered by a mobile-cellular networ...
4,High-income,2015,Total,98.5,Population covered by a mobile-cellular networ...
...,...,...,...,...,...
5955,Americas,2024,Rural,30.7,"Percentage of individuals using the Internet, ..."
5956,Arab States,2024,Rural,0.8,"Percentage of individuals using the Internet, ..."
5957,Asia-Pacific,2024,Rural,41.0,"Percentage of individuals using the Internet, ..."
5958,CIS,2024,Rural,0.0,"Percentage of individuals using the Internet, ..."


In [60]:
df_dvpmnt = pd.read_csv(r"output/dvpmnt_status_spec_reg_flat.csv")
df_dvpmnt

,Region,Year,Category,Value,Criteria
0,World,2005,Millions,1243.2,Fixed-telephone subscriptions
1,Least Developed Countries (LDCs),2005,Millions,6.4,Fixed-telephone subscriptions
2,World,2005,Millions,219.6,Fixed-telephone subscriptions
3,Least Developed Countries (LDCs),2005,Millions,0.0,Fixed-telephone subscriptions
4,World,2005,Millions,2205.3,Fixed-telephone subscriptions
...,...,...,...,...,...
14155,Upper-middle-income,2024,Total,84.2,Individuals owning a mobile phone***
14156,High-income,2024,Total,96.1,Individuals owning a mobile phone***
14157,Least Developed Countries (LDCs),2024,Total,62.8,Individuals owning a mobile phone***
14158,Land Locked Developing Countries (LLDCs),2024,Total,64.6,Individuals owning a mobile phone***


In [61]:
df_bdt = pd.read_csv(r"output/bdt_region_flat.csv")
df_bdt

,Region,Year,Category,Value,Criteria
0,Africa,2005,Millions,10.3,Fixed-telephone subscriptions/Millions
1,Americas,2005,Millions,290.9,Fixed-telephone subscriptions/Millions
2,Arab States,2005,Millions,29.6,Fixed-telephone subscriptions/Millions
3,Asia-Pacific,2005,Millions,559.1,Fixed-telephone subscriptions/Millions
4,CIS,2005,Millions,51.6,Fixed-telephone subscriptions/Millions
...,...,...,...,...,...
15269,Americas,2024,Total,88.8,Individuals owning a mobile phone***(Millions)
15270,Arab States,2024,Total,83.1,Individuals owning a mobile phone***(Millions)
15271,Asia-Pacific,2024,Total,76.9,Individuals owning a mobile phone***(Millions)
15272,CIS,2024,Total,94.4,Individuals owning a mobile phone***(Millions)


In [62]:
df_all = pd.concat(
    [df_gender, def_age, df_urban_rural, df_dvpmnt, df_bdt],
    axis=0,          # empiler les lignes
    ignore_index=True  # réindexer de 0 à n-1
)

In [63]:
df_all.to_csv("./output/all_data.csv", index=False)

In [64]:
df_all

,Region,Year,Category,Value,Criteria
0,World,2019,Total,52.9,"Percentage of individuals using the Internet, ..."
1,Low-income,2019,Total,17.2,"Percentage of individuals using the Internet, ..."
2,Lower-middle-income,2019,Total,33.5,"Percentage of individuals using the Internet, ..."
3,Upper-middle-income,2019,Total,64.5,"Percentage of individuals using the Internet, ..."
4,High-income,2019,Total,87.5,"Percentage of individuals using the Internet, ..."
...,...,...,...,...,...
36197,Americas,2024,Total,88.8,Individuals owning a mobile phone***(Millions)
36198,Arab States,2024,Total,83.1,Individuals owning a mobile phone***(Millions)
36199,Asia-Pacific,2024,Total,76.9,Individuals owning a mobile phone***(Millions)
36200,CIS,2024,Total,94.4,Individuals owning a mobile phone***(Millions)


In [65]:
df_all.duplicated().sum()


1232

In [66]:
df_all[df_all.duplicated()]


,Region,Year,Category,Value,Criteria
522,CIS,2020,Total,79.5,Percentage of individuals owning a mobile phon...
531,Small Island Developing States (SIDS),2020,Total,73.8,Percentage of individuals owning a mobile phon...
539,Americas,2021,Total,81.9,Percentage of individuals owning a mobile phon...
1453,Low-income,2015,Urban,13.4,Population covered by a mobile-cellular networ...
1484,High-income,2016,Urban,100.0,Population covered by a mobile-cellular networ...
...,...,...,...,...,...
35236,Individuals using the Internet(Millions),2023,Total,2023.0,Mobile broadband traffic(Exabytes)
35243,Individuals owning a mobile phone***(Millions),2023,Total,2023.0,Mobile broadband traffic(Exabytes)
35256,International bandwidth usage(Tbits/s**),2024,Total,2024.0,Mobile broadband traffic(Exabytes)
35262,Individuals using the Internet(Millions),2024,Total,2024.0,Mobile broadband traffic(Exabytes)


In [67]:
sorted(df_all["Region"].unique()[:50])

['Active mobile-broadband subscriptions/Millions',
 'Africa',
 'Americas',
 'Arab States',
 'Asia-Pacific',
 'CIS',
 'Europe',
 'Fixed broadband traffic(Exabytes)',
 'Fixed-broadband subscriptions/Millions',
 'High-income',
 'Individuals owning a mobile phone***(Millions)',
 'Individuals using the Internet(Millions)',
 'International bandwidth usage(Tbits/s**)',
 'Land Locked Developing Countries (LLDCs)',
 'Least Developed Countries (LDCs)',
 'Low-income',
 'Lower-middle-income',
 'Mobile broadband traffic(Exabytes)',
 'Mobile-cellular telephone subscriptions/Millions',
 'Population covered by a mobile-cellular network/Millions',
 'Population covered by at least a 3G mobile network/Millions',
 'Population covered by at least a 5G mobile network/Millions',
 'Population covered by at least an LTE/WiMAX mobile network/Millions',
 'Small Island Developing States (SIDS)',
 'Upper-middle-income',
 'World']

In [68]:
valid_regions = [
    # Régions ITU
    "Africa",
    "Americas",
    "Arab States",
    "Asia-Pacific",
    "CIS",
    "Europe",
    "World",
    
    # Groupes de revenus
    "High-income",
    "Low-income",
    "Lower-middle-income",
    "Upper-middle-income",
    
    # Groupes spéciaux
    "Least Developed Countries (LDCs)",
    "Land Locked Developing Countries (LLDCs)",
    "Small Island Developing States (SIDS)",
]
df_all = df_all[df_all["Region"].isin(valid_regions)].copy()


In [69]:
df_all.duplicated().sum()
df_all[df_all.duplicated()]


,Region,Year,Category,Value,Criteria
522,CIS,2020,Total,79.5,Percentage of individuals owning a mobile phon...
531,Small Island Developing States (SIDS),2020,Total,73.8,Percentage of individuals owning a mobile phon...
539,Americas,2021,Total,81.9,Percentage of individuals owning a mobile phon...
1453,Low-income,2015,Urban,13.4,Population covered by a mobile-cellular networ...
1484,High-income,2016,Urban,100.0,Population covered by a mobile-cellular networ...
...,...,...,...,...,...
27116,Africa,2015,Total,11.1,Mobile-cellular telephone subscriptions/Millions
28885,Africa,2015,Total,11.1,Active mobile-broadband subscriptions/Millions
30434,Africa,2015,Percentage,11.1,Population covered by a mobile-cellular networ...
31703,Africa,2015,Total,11.1,Population covered by at least a 3G mobile net...


In [70]:
df_all = df_all.drop_duplicates()
df_all

,Region,Year,Category,Value,Criteria
0,World,2019,Total,52.9,"Percentage of individuals using the Internet, ..."
1,Low-income,2019,Total,17.2,"Percentage of individuals using the Internet, ..."
2,Lower-middle-income,2019,Total,33.5,"Percentage of individuals using the Internet, ..."
3,Upper-middle-income,2019,Total,64.5,"Percentage of individuals using the Internet, ..."
4,High-income,2019,Total,87.5,"Percentage of individuals using the Internet, ..."
...,...,...,...,...,...
36197,Americas,2024,Total,88.8,Individuals owning a mobile phone***(Millions)
36198,Arab States,2024,Total,83.1,Individuals owning a mobile phone***(Millions)
36199,Asia-Pacific,2024,Total,76.9,Individuals owning a mobile phone***(Millions)
36200,CIS,2024,Total,94.4,Individuals owning a mobile phone***(Millions)
